In [32]:
import pandas as pd
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor, as_completed
from aspiratio.utils.report_downloader import find_annual_reports

repo = Path('..').resolve()
coverage_path = repo / 'coverage_table_updated.csv'
validation_path = repo / 'validation_results.csv'

coverage = pd.read_csv(coverage_path, sep='\t')
years = list(range(2019, 2025))

# Load validation results to identify already-validated reports
validated_df = pd.read_csv(validation_path)
validated_set = set(
    (row['CID'], row['Year']) 
    for _, row in validated_df[validated_df['Valid'] == True].iterrows()
)
print(f"✓ Already validated: {len(validated_set)} company-year combinations")

# Filter to incomplete entries (not Complete ✓)
incomplete = coverage[coverage['Priority'] != 'Complete ✓']

# Further filter out any company-year combos that are already validated
def needs_search(row):
    return (row['Company_Identifier'], row['FiscalYear']) not in validated_set

incomplete_needing_search = incomplete[incomplete.apply(needs_search, axis=1)]
print(f"→ Incomplete in coverage: {len(incomplete)}")
print(f"→ Actually need to search: {len(incomplete_needing_search)}")

incomplete_needing_search[['CompanyName', 'Company_Identifier', 'FiscalYear', 'IR_URL']].head(10)

✓ Already validated: 98 company-year combinations
→ Incomplete in coverage: 85
→ Actually need to search: 49


,CompanyName,Company_Identifier,FiscalYear,IR_URL
12,Boliden,S7,2019,https://investors.boliden.com/
49,Lifco B,S17,2020,https://www.lifco.se/investors/financial-reports
54,NIBE Industrier B,S18,2019,https://www.nibe.com/investors
55,NIBE Industrier B,S18,2020,https://www.nibe.com/investors
56,NIBE Industrier B,S18,2021,https://www.nibe.com/investors
57,NIBE Industrier B,S18,2022,https://www.nibe.com/investors
58,NIBE Industrier B,S18,2023,https://www.nibe.com/investors
78,SEB A,S23,2019,https://sebgroup.com/investor-relations
79,SEB A,S23,2020,https://sebgroup.com/investor-relations
80,SEB A,S23,2021,https://sebgroup.com/investor-relations


In [33]:
# Build search tasks: group by company and collect missing years
search_tasks = {}
for company, group in incomplete_needing_search.groupby(['CompanyName', 'Company_Identifier', 'IR_URL']):
    name, cid, ir_url = company
    target_years = sorted(group['FiscalYear'].unique().tolist())
    search_tasks[(name, cid, ir_url)] = target_years

print(f"🔍 Search tasks: {len(search_tasks)} companies")
for (name, cid, _), yrs in list(search_tasks.items())[:5]:
    print(f"   {name} ({cid}): years {yrs}")

🔍 Search tasks: 10 companies
   Boliden (S7): years [2019]
   Hennes & Mauritz B (S13): years [2020, 2022, 2023, 2024]
   Industrivärden C (S15): years [2019, 2020, 2021, 2022, 2023, 2024]
   Lifco B (S17): years [2020]
   NIBE Industrier B (S18): years [2019, 2020, 2021, 2022, 2023]


In [34]:
# Search for annual reports in parallel with Cision and MFN fallbacks
import importlib
import aspiratio.utils.report_downloader as rd_module
import aspiratio.tier1.mfn_search as mfn_module
import aspiratio.tier1.cision_search as cision_module

# Reload modules to pick up any changes
importlib.reload(mfn_module)
importlib.reload(cision_module)

from aspiratio.utils.report_downloader import find_annual_reports
from aspiratio.tier1.mfn_search import find_reports_via_mfn
from aspiratio.tier1.cision_search import find_reports_via_cision

def search_company(args):
    """Worker function to search one company with Cision and MFN fallbacks."""
    name, cid, ir_url, target_years = args
    results = []
    
    # Try standard IR search first
    try:
        candidates = find_annual_reports(ir_url, years=target_years, company_name=name)
        for c in candidates:
            results.append({
                'Company_Identifier': cid,
                'CompanyName': name,
                'Year': c.get('year'),
                'URL': c.get('url'),
                'Source_Page': c.get('source_page', ir_url),
                'Title': c.get('title', ''),
                'Source': c.get('source', 'ir')
            })
    except Exception as e:
        print(f"    ⚠ IR search error for {name}: {e}")
    
    # Check which years are still misOne thing that I'm a little bit curious about in this specific step it is uh cell number three. So when it's looking for the annual reports.sing
    found_years = {r['Year'] for r in results}
    missing_years = [y for y in target_years if y not in found_years]
    
    # Fallback 1: Try Cision News (primary source for Swedish press releases)
    if missing_years:
        print(f"    🔄 {name}: Missing years {missing_years}, trying Cision fallback...")
        try:
            cision_candidates = find_reports_via_cision(name, missing_years)
            for c in cision_candidates:
                if c.get('year') in missing_years:  # Only add if still missing
                    results.append({
                        'Company_Identifier': cid,
                        'CompanyName': name,
                        'Year': c.get('year'),
                        'URL': c.get('url'),
                        'Source_Page': c.get('source_page', ''),
                        'Title': c.get('title', ''),
                        'Source': 'cision'
                    })
                    if c.get('year') in missing_years:
                        missing_years.remove(c.get('year'))
        except Exception as e:
            print(f"    ⚠ Cision fallback error for {name}: {e}")
    
    # Fallback 2: Try MFN if still missing years
    if missing_years:
        print(f"    🔄 {name}: Still missing {missing_years}, trying MFN fallback...")
        try:
            mfn_candidates = find_reports_via_mfn(name, missing_years)
            for c in mfn_candidates:
                results.append({
                    'Company_Identifier': cid,
                    'CompanyName': name,
                    'Year': c.get('year'),
                    'URL': c.get('url'),
                    'Source_Page': c.get('source_page', ''),
                    'Title': c.get('title', ''),
                    'Source': 'mfn'
                })
        except Exception as e:
            print(f"    ⚠ MFN fallback error for {name}: {e}")
    
    status = 'success' if results else 'no_results'
    return name, cid, status, results

# Prepare tasks
task_list = [
    (name, cid, ir_url, yrs) 
    for (name, cid, ir_url), yrs in search_tasks.items()
]

# Run searches in parallel (max 5 concurrent to avoid overwhelming servers)
report_candidates = []
max_workers = min(5, len(task_list)) if task_list else 1

print(f"🚀 Starting parallel search with {max_workers} workers...")
print(f"   Companies to search: {len(task_list)}")
print(f"   Fallback chain: IR → Cision → MFN")

with ThreadPoolExecutor(max_workers=max_workers) as executor:
    futures = {executor.submit(search_company, task): task for task in task_list}
    
    for future in as_completed(futures):
        name, cid, status, results = future.result()
        if results:
            print(f"  ✓ {name} ({cid}): found {len(results)} candidates")
        else:
            print(f"  ✗ {name} ({cid}): no candidates found")
        report_candidates.extend(results)

print(f"\n📊 Total candidates found: {len(report_candidates)}")
candidates_df = pd.DataFrame(report_candidates)
candidates_df.head(10)

🚀 Starting parallel search with 5 workers...
   Companies to search: 10
   Fallback chain: IR → Cision → MFN
Searching: https://investors.boliden.com/
Searching: https://hmgroup.com/investors/
Searching: https://www.industrivarden.se/en-gb/investors/
Searching: https://www.lifco.se/investors/financial-reports
Searching: https://www.nibe.com/investors
  → Found 40 links to analyze (from HTML, JSON, nav, footer)
  → Analyzed 2 PDF links, excluded 0 quarterly/SEC filings
  Searching: https://www.lifco.se/contact?lang=en
  → Found 1632 links to analyze (from HTML, JSON, nav, footer)
    ✓ Found report: 2020 - annual and sustainability report 2024
      URL: .../HM-Group-Annual-and-sustainability-repor
    ✓ Found report: 2020 - annual and sustainability report 2024
      URL: .../HM-Group-Annual-and-sustainability-repor
  → Analyzed 10 PDF links, excluded 0 quarterly/SEC filings
❌ HTTP 404 Not Found (failure 1/3)
  No reports found in standard search, trying common patterns...
Searching: h

,Company_Identifier,CompanyName,Year,URL,Source_Page,Title,Source
0,S17,Lifco B,2020,https://network.s-z.se/app/uploads/sites/16/Li...,https://www.lifco.se/investors/financial-repor...,,ir
1,S13,Hennes & Mauritz B,2020,https://hmgroup.com/wp-content/uploads/2025/03...,https://hmgroup.com/investors/,annual and sustainability report 2024,ir
2,S13,Hennes & Mauritz B,2022,https://mb.cision.com/Main/769/3743344/1953371...,https://news.cision.com/se/h---m-hennes---maur...,h & m hennes & mauritz ab:s års- och hållbarhe...,cision
3,S13,Hennes & Mauritz B,2023,https://mb.cision.com/Main/769/3952727/2698038...,https://news.cision.com/se/h---m-hennes---maur...,h & m hennes & mauritz ab:s års- och hållbarhe...,cision
4,S13,Hennes & Mauritz B,2024,https://mb.cision.com/Main/769/4124721/3347739...,https://news.cision.com/se/h---m-hennes---maur...,h & m hennes & mauritz ab publicerar sin års- ...,cision
5,S19,Nordea Bank Abp,2024,https://www.nordea.com/en/doc/annual-report-no...,https://www.nordea.com/en/investors/latest-ann...,read ceo comment,ir
6,S19,Nordea Bank Abp,2019,https://mb.cision.com/Main/434/3047655/1201947...,https://news.cision.com/nordea/r/nordea-has-pu...,"nordea has published its annual report, sustai...",cision
7,S19,Nordea Bank Abp,2021,https://mb.cision.com/Public/434/3517156/a6765...,https://news.cision.com/nordea/r/nordea-has-pu...,nordea has published its annual reporting pack...,cision
8,S19,Nordea Bank Abp,2022,https://mb.cision.com/Public/434/3725516/812c3...,https://news.cision.com/nordea/r/nordea-has-pu...,nordea has published its annual reporting for ...,cision
9,S19,Nordea Bank Abp,2023,https://mb.cision.com/Main/434/3935418/2626561...,https://news.cision.com/nordea/r/nordea-has-pu...,nordea has published its annual reporting for ...,cision


In [35]:
# Persist candidates for review (only if we found any)
if not candidates_df.empty:
    output = repo / 'report_candidates.csv'
    candidates_df.to_csv(output, index=False)
    print(f"✓ Saved {len(candidates_df)} candidates to {output}")
else:
    print("⚠ No candidates found - nothing to save")

✓ Saved 22 candidates to /Users/jakobjohannesson/Documents/github_repos/aspiratio/report_candidates.csv
